In [2]:
print('hello')

hello


In [ ]:
#!/usr/bin/env python3
# ------------------------------------------------------------
#  gpt2-ppo で微調整したモデルをロードして文章を生成する例
# ------------------------------------------------------------
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

# 1) 保存先ディレクトリ
CKPT_DIR = Path("D:\github\BISMARL-LLM\checkpoints\gpt2_ppo_final")   # 相対パスでも絶対パスでも OK

# 2) デバイス選択（GPU があれば GPU を使う）
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] using device: {device}")

# 3) トークナイザー & モデルをロード
tokenizer = AutoTokenizer.from_pretrained(CKPT_DIR)
model     = AutoModelForCausalLM.from_pretrained(CKPT_DIR).to(device)
tokenizer.padding_side = "left"                   # 学習時と合わせる
tokenizer.pad_token_id = tokenizer.eos_token_id   # 忘れずに

# 4) 乱数シードを固定（再現性が欲しい場合）
set_seed(123)

# 5) 生成用ハイパーパラメータ（学習時の設定に近い値）
gen_kwargs = dict(
    max_new_tokens    = 64,
    min_length        = 0,
    top_p             = 0.9,
    top_k             = 40,
    temperature       = 0.7,
    do_sample         = True,
    repetition_penalty= 1.1,
    pad_token_id      = tokenizer.eos_token_id,
)

# 6) 生成ループ（Ctrl-C で終了）
print("\n=== Interactive generation (type a prompt and hit Enter) ===")
try:
    while True:
        prompt = input("\nUser > ").strip()
        if not prompt:
            continue

        # トークン化 & 生成
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
        with torch.no_grad():
            output_ids = model.generate(input_ids, **gen_kwargs)

        # 生成された部分だけ取り出し & 表示
        generated = tokenizer.decode(output_ids[0][input_ids.size(1):], skip_special_tokens=True)
        print(f"Model> {generated}")
except (KeyboardInterrupt, EOFError):
    print("\n[INFO] Bye! 👋")


c:\Users\ayato\anaconda3\envs\CORY\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] using device: cuda


OSError: Incorrect path_or_model_id: 'checkpoints\gpt2_ppo_final'. Please provide either the path to a local folder or the repo_id of a model on the Hub.